<a id='start'></a>
# Esercizi Reti neurali con Tensor Flow

In questo notebook vengono presentati degli esercizi sulle reti neurali con Tensor Flow.

Provate a svolgere il seguente esercizio:<br>
1) [pp -> H -> ZZ -> 4lepton](#section1)<br>
2) [Classificare i getti di particelle](#section2)<br>

<a id='section1'></a>
## pp -> H -> ZZ -> 4lepton

Creare una rete neurale per analizzare i dati prodotti dagli studi di pp -> H -> ZZ -> 4lepton nel contesto delle ricerche mono-Higgs da parte della collaborazione CMS. I programmi usano le ntuple ridotte (con il nome dell'albero radice HZZ4LeptonsAnalysisReduced) create con Root.

In [27]:
import numpy as np

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

treename = 'HZZ4LeptonsAnalysisReduced'

VARS = ['f_mass4l', 'f_massjj']
upfile_VV = uproot.open('data/ntuple_4mu_VV.root')
params_VV = upfile_VV[treename].arrays(VARS, library="np")
upfile_BKG = uproot.open('data/ntuple_4mu_bkg.root')
params_BKG = upfile_BKG[treename].arrays(VARS, library="np")

df_VV = pd.DataFrame(params_VV, columns=VARS)
df_BKG = pd.DataFrame(params_BKG, columns=VARS)
    
# cut out undefined variables VARS[0] and VARS[1] > -999
df_VV = df_VV[(df_VV[VARS[0]] > -999) & (df_VV[VARS[1]] > -999)]
df_BKG = df_BKG[(df_BKG[VARS[0]] > -999) & (df_BKG[VARS[1]] > -999)] 

# add isSignal variable
df_VV['isSignal'] = np.ones(len(df_VV)) 
df_BKG['isSignal'] = np.zeros(len(df_BKG))

df_all = pd.concat([df_VV, df_BKG])
df_all

,f_mass4l,f_massjj,isSignal
0,125.077103,1300.426880,1.0
1,124.238113,437.221863,1.0
3,124.480667,1021.744080,1.0
4,124.919464,1101.381958,1.0
7,125.049065,498.717194,1.0
...,...,...,...
58056,197.383133,114.257278,0.0
58057,218.065582,106.195534,0.0
58060,89.288086,53.661579,0.0
58062,146.756577,71.162025,0.0


Creiamo una rete densa, con un singolo livello nascosto completamente connesso con lo stesso numero di neuroni delle variabili di input.

Possiamo quindi usare la funzione di attivazione `sigmoide` per produrre un'uscita di probabilità nell'intervallo da 0 a 1.

Infine usiamo la funzione di loss `binary_crossentropy` durante l'allenamento, una funzione di perdita standard per problemi di classificazione binaria.

Infine utimmizziamo il modello con l'algoritmo Adam per il grandient descend.

<a id='section2'></a>
## Classificare i getti di particelle

Caricare i dati da `jetImage.h5` e quindi definizimo una DNN con tre livelli che, una volta allenata, aiuti a classificare i getti di particelle contenuti in jets.

Le ultime 6 feature (**'j_g' 'j_q' 'j_w' 'j_z' 'j_t' 'j_undef'**) riportano l'informazione di quale jet stiamo osservando. Le informazioni sono fornite con una codifica one-hot: tutti i flag sono 0 tranne quello per la categoria corretta, che è impostato su 1.

Ad esempio, un **W-jet** avrà indici **[0,0,1,0,0,0]**

Si noti che l'ultimo flag (**undefined**) viene utilizzato per jet di categorie diverse dalle cinque specificate. Poiché il nostro set di dati ha solo getti di questi cinque tipi, questo flag è sempre impostato su 0 e può essere scartato.

In [2]:
import h5py
import numpy as np

f = h5py.File("data/jetImage.h5")
jets = f.get('jets')
y = np.array(jets[:,-6:-1])
X = np.array(jets[:,:-6])

[Clicca qui per tornare all'inizio della pagina](#start)<a id='start'></a>